# T-prime and T-ref calculation

This notebook will guide you through the processing of $ T_{ref} $ and $ T_{p} $ on both reanalysis and CIMP6 models. The process relies heavily on Dask `job-queue` and uses all the power of RCCs Midway to parallelize the code and get fast calculations. Be mindful that to reproduce this code you need to correctly configure `job-queue` by changing the `jobqueue.yaml` file (usually is in this path: in `$HOME/.config/dask/`). 

A small example of this configuration file is this one: 

```yaml
jobqueue:
  slurm:
    name: dask-worker

    # Dask worker options
    cores: 28                  # Total number of cores per job
    memory: '64GB'             # Total amount of memory per job
    processes: 1               # Number of Python processes per job
    n_workers: 0
    nanny: false
    job_extra:
        - '--account=geos39650'
        - '--output=dask_worker.out'
        - '--error=dask_worker.err'

    interface: ib0
    local-directory: $SCRATCH$ 
    queue: 'broadwl'
    walltime: '00:10:00'
```

Here, we're requesting 28 cores and 20 GB of RAM memory per job, this conincides with the average size of nodes in the `broadwl` partition. Every task we create will be pushed to each node! The number of requested nodes is dynamic, and can be either be user-defined or by-demand using the `cluster`. We will see how this works later! 


### Scheduler cool visuals

Dask `job-queue` comes with a powerful shceduler visualizer that gives the user some hints about the job progress and efficiency of the cluster (in escence, Dask is actually creating its own cluster and submitting dedicated dask operations to it). The sheduler runs by default in the port 8484, and unless used, it can be accessed using from `https://localhost:8787`. But not so fast! This is running in the login node (if you're running this from the login node), so you need to run a `ssh` tunnel for that: `ssh -N -L 8787:<client_ip>:8787 ivanhigueram@midway.rcc.uchicago.edu`. Where `<client_ip>` is the IP address returned by the `Client` object.


Let us load the libraries we will use:

In [1]:
import os
import dask
import xarray as xr
from pathlib import Path
from dask_jobqueue import SLURMCluster
from dask.distributed import Client


from methods import (t_prime_calculation, 
                     dask_data_to_xarray,
                     area_calculation_real_area,
                     dists_of_lat_eff,
                     temp_ref)

### Set up cluster
As discussed earlier, we need to load our cluster configuration stored in the `jobqueue.yaml` configuration file. We can print the job request by running `cluster.job_script()`. Here we need two parts. First, we need a cluster object with all the parameters to pass to the Slurm manager. Second, we need a client that will be use by `xarray` and `dask` operations.

In [2]:
cluster = SLURMCluster(nanny=False)
print(cluster.job_script())
client = Client(cluster)

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p broadwl
#SBATCH -n 1
#SBATCH --cpus-per-task=4
#SBATCH --mem=19G
#SBATCH -t 00:20:00
#SBATCH --account=geos39650
#SBATCH --output=dask_worker.out
#SBATCH --error=dask_worker.err

/scratch/midway2/ivanhigueram/reanalysis_env/bin/python -m distributed.cli.dask_worker tcp://172.25.220.71:48162 --nthreads 4 --memory-limit 20.00GB --name name --no-nanny --death-timeout 60 --local-directory $SCRATCH --interface ib0



As seen above, we will request the above computing power everytime we decide to scale our `client` object. By default, our client will have 0 workers, this to avoid requesting for un-used services and to have total control over the computing requests. The method `scale` allows to scale computation in a simple way. By default, any positive integer passed to `scale` will request that equivalent number of workers. We can request less resources than a complete node. 

In [3]:
# There's no rule-of-thumb in deciding how to decide the number of clusters. All depends on the type of job you're running
cluster.scale(1)

In [3]:
client

Client Scheduler: tcp://172.25.220.71:48162 Dashboard: http://172.25.220.71:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


# Computation on different climate products 

**WARNING**: System administrators do not like users running heavy computations in cluster. 

In [7]:
#client.upload_file('methods.py')

## 1. Analysis with ERA-5 Reanalysis data

Using the `xarray` library, let's analyze the reanalysis product. We have a daily file of winter data in a daily resolution: `data/processed_winters/df_lat_20_1D.nc` [This file was generated with our code in `src/analyzer.py`]. 

For this processing we will execute everything inside the `calculate_t_prime` function and will let `job-queue` to decide how manyt workers between 0 and 10 it needs to do the job (usually takes the maximum). 

In [26]:
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

def calculate_t_prime(path_to_raw_file,
                      path_to_save,
                      start_year=1995,
                      end_year=2020,
                      size_year_range=5,
                      gcm_hack=False,
                      outputs=['t_ref', 't_prime'],
                      **kwargs):
    """
    Calculate t_prime for the user-defined path and output a NetCDF file with
    the processed data
    """

    print(f'Initializing t prime calculation')
    
    for year in range(start_year, end_year, size_year_range):
        # Define time ranges 
        start_year = datetime(year, 12, 1).strftime('%Y-%m-%d')
        end_year = datetime(year + size_year_range, 3, 1).strftime('%Y-%m-%d')

        print(f"Start processing -- {start_year} to {end_year}")
        xarray_file = xr.open_dataset(path_to_raw_file,
                                      chunks={'time': 1})
      
        
        if gcm_hack:
            model='gcm'
            xarray_file = xarray_file.isel(member_id=0, nbnd=0)
    
            datetime_index = xarray_file.indexes['time'].to_datetimeindex()
            xarray_file['time'] = datetime_index
            
            xarray_file = xarray_file.sel(time=slice(start_year, end_year))
            t_df = xarray_file.to_dask_dataframe(dim_order=['time', 'lat', 'lon'])
            t_df_renamed = t_df.rename(columns={
                    'lat': 'latitude',
                    'lon': 'longitude',
                    'tas': 't2m'
                })
            t_df = t_df_renamed[['time', 'latitude', 'longitude', 't2m']]  
        else:
            model='reanalysis'
            xarray_file_year_range = xarray_file.sel(time=slice(start_year, end_year))
            t_df = xarray_file_year_range.to_dask_dataframe(dim_order=['time', 'latitude', 'longitude'])

        # Initialize t_prime calculation using dask DataFrame
        test_ultimate = t_prime_calculation(t_df,
                                            grid_lat=kwargs['grid_lat'],
                                            grid_lon=kwargs['grid_lon'],
                                            resample_time=False,
                                            build_buckets=True,
                                            cut_interval=2
                                           )

        # Save results to NetCDF
        for out in outputs:

            print(f'Saving {out} file to netcdf')
            t_ref_test = dask_data_to_xarray(df=test_ultimate,
                                             dims=['time', 'latitude', 'longitude'],
                                             target_variable=out)
            path_save = os.path.join(path_to_save, f'{out}_gcm_{start_year}_{end_year}.nc4')
            t_ref_test.to_netcdf(path_save)

Send parallel jobs using `cluster.adapt`

In [19]:
#cluster.adapt(minimum=1, maximum=10)
cluster.scale(30)

In [ ]:
calculate_t_prime(path_to_raw_file='/project2/geos39650/jet_stream/data/processed_winters/df_lat_20_1D.nc',
                  path_to_save='/project2/geos39650/jet_stream/data/model_output/',
                  outputs=['t_prime', 't_ref'],
                  grid_lat=0.25,
                  grid_lon=0.25)

## 2. Analysis with CIMP6 climate projection data

Just as we did with the ERA-5 data, we will load the processed winter data from the projection data: `data/processed_winters/ScenarioMIP_NCAR_CESM2-WACCM_ssp585_day.nc4` [This file was generated with our code in `src/analyzer.py`]. 

For this processing we will execute everything inside the `calculate_t_prime` function and will let `job-queue` to decide how manyt workers between 0 and 10 it needs to do the job (usually takes the maximum). 

In [29]:
#cluster.adapt(minimum=30, maximum=40)
cluster.scale(30)

In [ ]:
calculate_t_prime(path_to_raw_file='/project2/geos39650/jet_stream/data/processed_winters/ScenarioMIP_NCAR_CESM2-WACCM_ssp585_day.nc4',
                  path_to_save='/project2/geos39650/jet_stream/data/model_output/',
                  grid_lat=0.9,
                  grid_lon=1.25,
                  start_year=2035,
                  end_year=2101,
                  size_year_range=10,
                  gcm_hack=True)

Initializing t prime calculation
Start processing -- 2035-12-01 to 2045-03-01


/scratch/midway2/ivanhigueram/reanalysis_env/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.


Saving t_ref file to netcdf
Saving t_prime file to netcdf
